# Tracking project example

This series of Notebooks will take you through the main steps to train a U-net to track objects in images.

As a toy example, we will track human fingers in a video. You can download the video here:  

https://drive.google.com/file/d/1ezrMsINCKPN85O4H71IKKd5cD8u4CE2M/view?usp=drive_link 

The notebook uses several Python objects from the unetTracker package. You can find the code related to these objects in the folder `unetTracker` in the unetTracker repository.


# Creating a tracking project

The configuration and dataset for your project will be saved in a directory. For instance, you could put your project directory in your `Documents` folder. You can also put this series of Notebooks in a folder within your project directory. This way, your notebooks will be kept with your model.

The `unetTracker.trackingProject.TrackingProject` object will load the configuration from your project directory.

The project directory will be several 100 Mb after you label many images used to train your model.

The configuration is saved to and loaded from a file called `config.yalm`. This file is located in your project directory. You can edit this file manually if needed and reload the configuration from the file.

You only need to run this notebook once when you create your project. If you have done it before, skip this notebook.

If you create a TrackingProject and give an `object_list`, the codes assumes that you want to create a new project.

In [1]:
from unetTracker.trackingProject import TrackingProject
import os

You can create a directory where you will store your unetTracker projects. 


In [2]:
root_path = "/home/kevin/Documents/trackingProjects"
if not os.path.exists(root_path):
    print("creating",root_path)
    os.makedirs(root_path)
else:
    print(root_path, "already exists")
    print("If you have already created your project in the past and adjusted the configuration, you don't need to run this notebook again")

/home/kevin/Documents/trackingProjects already exists
If you have already created your project in the past and adjusted the configuration, you don't need to run this notebook again


You can now create a TrackingProject object.

Set a meaningful name for your project and list the object/body parts that you want to track. I usually track as few objects as required.

The radius is determining the size of the markers that the model will try to track. It can be changed later on.

In [3]:
project = TrackingProject(name="finger_tracker2",
                          root_folder = root_path,
                          object_list=["f1","f2","f3","f4"],
                          target_radius=6)

Project directory: /home/kevin/Documents/trackingProjects/finger_tracker2


You can now create the project directories and save a default configuration file.

## Warning

If you already have a project directory and have edited the configuration file, running the cell below will overwrite your old configuration.

If you are working with a new project, go ahead and create directories and save the default configuration.

In [4]:
project.create_project_directories()
project.save_configuration()

Create /home/kevin/Documents/trackingProjects/finger_tracker2
Create /home/kevin/Documents/trackingProjects/finger_tracker2/dataset
Create /home/kevin/Documents/trackingProjects/finger_tracker2/dataset/images
Create /home/kevin/Documents/trackingProjects/finger_tracker2/dataset/coordinates
Create /home/kevin/Documents/trackingProjects/finger_tracker2/dataset/masks
Create /home/kevin/Documents/trackingProjects/finger_tracker2/models
Create /home/kevin/Documents/trackingProjects/finger_tracker2/augmentation
Saving /home/kevin/Documents/trackingProjects/finger_tracker2/config.yalm


You can inspect at your project directory. 

In [5]:
print("Project directory:", project.project_dir)
print("Files and subdirectories inside your project directory:",os.listdir(project.project_dir))

Project directory: /home/kevin/Documents/trackingProjects/finger_tracker2
Files and subdirectories inside your project directory: ['models', 'augmentation', 'dataset', 'config.yalm']


## Edit your config.yalm file

You can now edit the `config.yalm` file. You can use your favorite text editor to inspect or edit your `config.yalm`.

When you are done you can load the new configuration from file.

Make sure you set the correct size for your images. If you are using the `human_hand.mp4`, the image size is 270 by 480

* image_size (first value is height, second value is width)

The image_size will have a strong effect on how fast the model will run. Larger images will be much slower to process. I would recommend using images that are not larger than 480x640.

Also, if you have body parts that are characterized by left/right, set `augmentation_HorizontalFlipProb: 0.0`.


In [6]:
project.load_configuration()

Loading /home/kevin/Documents/trackingProjects/finger_tracker2/config.yalm
{'augmentation_HorizontalFlipProb': 0.5, 'augmentation_RandomBrightnessContrastProb': 0.2, 'augmentation_RandomSizedCropProb': 1.0, 'augmentation_RotateProb': 0.3, 'image_extension': '.png', 'image_size': [270, 480], 'labeling_ImageEnlargeFactor': 2.0, 'name': 'finger_tracker2', 'normalization_values': None, 'object_colors': [(0.0, 0.0, 255.0), (255.0, 0.0, 0.0), (255.0, 255.0, 0.0), (240.0, 255.0, 255.0)], 'objects': ['f1', 'f2', 'f3', 'f4'], 'target_radius': 6, 'unet_features': [64, 128, 256, 512]}


The configuration that is in the `config.yalm` is stored in `project.configDict`

In [7]:
project.configDict

{'augmentation_HorizontalFlipProb': 0.5,
 'augmentation_RandomBrightnessContrastProb': 0.2,
 'augmentation_RandomSizedCropProb': 1.0,
 'augmentation_RotateProb': 0.3,
 'image_extension': '.png',
 'image_size': [270, 480],
 'labeling_ImageEnlargeFactor': 2.0,
 'name': 'finger_tracker2',
 'normalization_values': None,
 'object_colors': [(0.0, 0.0, 255.0),
  (255.0, 0.0, 0.0),
  (255.0, 255.0, 0.0),
  (240.0, 255.0, 255.0)],
 'objects': ['f1', 'f2', 'f3', 'f4'],
 'target_radius': 6,
 'unet_features': [64, 128, 256, 512]}

## Creating a TrackingProject object from an existing project config.yalm file

The next time you want to create a TrackingProject object for your existing project, you can simply point to the existing project directory.

Because there is no object list given as argument, the code will look for an existing project and automatically load the configuration from file.

This is how you will create the TrackingProject object in the following notebooks.

In [8]:
model_name = "finger_tracker2"
root_path = "/home/kevin/Documents/trackingProjects"
project = TrackingProject(name=model_name,root_folder = root_path)

Project directory: /home/kevin/Documents/trackingProjects/finger_tracker2
Getting configuration from config file. Values from config file will be used.
Loading /home/kevin/Documents/trackingProjects/finger_tracker2/config.yalm
{'augmentation_HorizontalFlipProb': 0.5, 'augmentation_RandomBrightnessContrastProb': 0.2, 'augmentation_RandomSizedCropProb': 1.0, 'augmentation_RotateProb': 0.3, 'image_extension': '.png', 'image_size': [270, 480], 'labeling_ImageEnlargeFactor': 2.0, 'name': 'finger_tracker2', 'normalization_values': None, 'object_colors': [(0.0, 0.0, 255.0), (255.0, 0.0, 0.0), (255.0, 255.0, 0.0), (240.0, 255.0, 255.0)], 'objects': ['f1', 'f2', 'f3', 'f4'], 'target_radius': 6, 'unet_features': [64, 128, 256, 512]}


In [9]:
project.configDict

{'augmentation_HorizontalFlipProb': 0.5,
 'augmentation_RandomBrightnessContrastProb': 0.2,
 'augmentation_RandomSizedCropProb': 1.0,
 'augmentation_RotateProb': 0.3,
 'image_extension': '.png',
 'image_size': [270, 480],
 'labeling_ImageEnlargeFactor': 2.0,
 'name': 'finger_tracker2',
 'normalization_values': None,
 'object_colors': [(0.0, 0.0, 255.0),
  (255.0, 0.0, 0.0),
  (255.0, 255.0, 0.0),
  (240.0, 255.0, 255.0)],
 'objects': ['f1', 'f2', 'f3', 'f4'],
 'target_radius': 6,
 'unet_features': [64, 128, 256, 512]}

## Adjust your setup_project.py file

At the beginning of all subsequent notebooks, we will load our project from the project directory we just created. 

To avoid having to remember all the details on how this is done, we can just put the code in a file called `setup_project.py` that is found in the same directory as the notebooks. At the begining of each subsequent notebook, we will execute this code.

Go ahead and adjust the `setup_project.py` so that its content points to your project directory. Once you are done, you can run the code below to load your project.

In [11]:
# this will run the code in the setup_project.py and create a variable called `project`
%run setup_project.py

Project directory: /home/kevin/Documents/trackingProjects/finger_tracker2
Getting configuration from config file. Values from config file will be used.
Loading /home/kevin/Documents/trackingProjects/finger_tracker2/config.yalm
{'augmentation_HorizontalFlipProb': 0.5, 'augmentation_RandomBrightnessContrastProb': 0.2, 'augmentation_RandomSizedCropProb': 1.0, 'augmentation_RotateProb': 0.3, 'image_extension': '.png', 'image_size': [270, 480], 'labeling_ImageEnlargeFactor': 2.0, 'name': 'finger_tracker2', 'normalization_values': None, 'object_colors': [(0.0, 0.0, 255.0), (255.0, 0.0, 0.0), (255.0, 255.0, 0.0), (240.0, 255.0, 255.0)], 'objects': ['f1', 'f2', 'f3', 'f4'], 'target_radius': 6, 'unet_features': [64, 128, 256, 512]}
